<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="150" width="150" align="right"/>

## Collaborative Filtering (2)
[Nacho Santos](www.nacho.santos.name)

## Import python packages

In [119]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [120]:
y = np.load('matrizy.npy')
r = np.load('matrizr.npy')

In [121]:
# Other functions necessary for this assignment
# The python file "recommender_system.py" must be in the same folder as this notebook, otherwise,
# you have to add the path to the file
from recommender_system import *

In [122]:
# This line is necessary to show matplotlib plots inside the jupyter notebook
%matplotlib inline

## 2 The cost funcion J and the gradient of J
The objective of this point is to build a function to compute the cost J and the corresponding gradient of J. In particular, you are going to implement a function called **cofiCostFunc()** with the arguments(inputs) and outputs detailed below (the code of the function is partially predefined in a cell right after).

**Arguments** (in this order)
* *parameters* (the paramaters of the cost function, i.e. X and $\theta$
* *Y* (matrix of ratings)
* *R* (matrix of watched movies)
* *n_users* (number of users)
* *n_movies* (number of movies)
* *n_characteristics* (number of the filter characteristics)
* *landa* (regularization parameter)

**Outputs** (in this order)
* *cost* (value of the cost function J)
* *gradient* (gradient of the cost function J)

In [123]:
# Cost and gradient function
n_features = 100
x_0 = np.random.random_sample((y.shape[0],n_features))
thetha_0 = np.random.random_sample((y.shape[1],n_features))
vecFlat_0 = np.append(x_0.flatten(),thetha_0.flatten())
n_users = y.shape[1]
n_movies = y.shape[0]

def cofiCostFunc (parameters, Y, R, n_users, n_movies, n_features, lamb):
# parameters: vector with the matrices X and Theta foldes
# Y: matrix of ratings
# R: matrix of watched movies
# n_users: number of users (number of columns of the matrix Y)
# n_movies: number of movies (number of rows of the matrix Y)
# n_features: number of movies' features (a parameter of the CF algorithm)
# lamb: regularization term
#
# cofiCostFunc rerturns the cost J and the gradient of J

    # You need to return the following values correctly
    cost=0
    gradient=np.zeros_like(parameters)
    
    # Remember:
    #  (1) To unfold X and Theta from parameters before computing J and the gradients
    #  (2) To fold the gradient of J with respect to X and to Theta into a flattened vector gradient 
    #      that is returned by the function
    
    
    # YOUR CODE ..................................
    
    x = parameters[0:(n_movies*n_features)].reshape((n_movies,n_features))
    thetha = parameters[(n_movies*n_features):].reshape((n_users,n_features))
    thethaT = thetha.transpose()  
    mul = np.dot(x,thethaT)   
    ycapuchino = np.multiply(mul,R)
    
    primera_parte_coste = (1/2)*np.sum((ycapuchino-Y)**2)
    segunda_parte_coste = (lamb/2)*np.sum((x)**2)
    tercera_parte_coste = (lamb/2)*np.sum((thetha)**2)
    
    cost = primera_parte_coste+segunda_parte_coste+tercera_parte_coste
    
    a = np.multiply(np.dot(x,thetha.T),R) - Y
    grad_thetha = np.dot(a,thetha) + lamb*x

    a = np.multiply(np.dot(x,thetha.T),R) - Y
    grad_x = np.dot(a.T,x) + lamb*thetha

    gradient = np.append(grad_thetha.flatten(),grad_x.flatten())
    
    
    
    

    # YOUR CODE (end) ..................................
    
    return (cost, gradient)

### 2.1 Input *parameters* of the cofiCostFunc

**Features X and preferences Theta**

The Collaborative Filtering (CF) algorithm is based on two sets of lineas regressions, the first one corresponds to the movies' features X, and the second one corresponds to the users' preferences Theta. Assuming n features, the matrix X will be:

$$X=\begin{bmatrix}x^{(1)}_{1} & ...& x^{(1)}_{n} \\. & ...& .\\x^{(m)}_{1} & ...& x^{(m)}_{n} \end{bmatrix}$$

where the i-th row of X corresponds to the feature vector $x^{(i)}$ for the i-th movie.

And the matrix Theta will be:

$$Theta=\Theta=\begin{bmatrix}\theta^{(1)}_{1} & ...& \theta^{(1)}_{n} \\. & ...& .\\\theta^{(u)}_{1} & ...& \theta^{(u)}_{n} \end{bmatrix}$$

where the j-th row of Theta corresponds to the preference vector $\theta^{(j)}$ for the j-th user. 

**Passing X and Theta to cofiCostFunc**

We are going to use a optimize package scipy.optimize that requires using a **flattened vector** of parameters. However, in our problem tha parameters to be optimized are represented by two matrices, i.e. X and Theta. So, X and Theta must be passed to the cofiCostFunc as a **(mxn)+(u+n) vector**, called **parameters**:

$${ \left[ \begin{matrix} { x }^{ (1) }, & ... & { x }^{ (m) }, \end{matrix}\begin{matrix} \theta ^{ (1) }, & ... & \theta ^{ (u) } \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$ 

However, inside the function, you can unfold the vector **parameters** and build the matrices X and Theta to compute J and the gradients according to the equations explained in class.

### 2.2 Computing the cost J
Suppose that the vector of features $x^{(i)}$ of the film i and the vector of preferences $\theta^{(j)}$ of the user j are known, then the **estimate of the rating** of the user j for the movie i will be:

$$\widehat{y}^{(i,j)}=x^{(i)}(\theta^{(j)})^{T}$$

The error of the estimate will be the difference between the estimate of rating $\widehat{y}^{(i,j)}$ and the real ratings $y^{(i,j)}$

The **cost J** is defined as the the average of the squares of the errors plus two regularization terms:

$$J=\frac { 1 }{ 2 } \sum _{ (i,j):r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) ^{ 2 } } +\quad \frac { \lambda  }{ 2 } \sum _{ i=1 }^{ m }{ \sum _{ k=1 }^{ n }{ ({ x }_{ k }^{ (i) })^{ 2 } }  } +\frac { \lambda  }{ 2 } \sum _{ j=1 }^{ u }{ \sum _{ k=1 }^{ n }{ ({ \theta  }_{ k }^{ (j) })^{ 2 } }  } $$


### Task 7
***
Implement the cost J as a vectorized expression (recommended). For example, the estimate of ratings can be expressed as:

$$\widehat{Y}=X\Theta^{T}$$

Now, go back and **complete the cofiCostFunc code to compute the cost J**. Remeber that J is scalar value.

### 2.3 Checking the cost J
Now, you will import a data set and check the cofiCostFunc.

In [124]:
# load dataset for checking
Y=np.load('YmatrixTest.npy')
R=np.load('RmatrixTest.npy')
X=np.load('XmatrixTest.npy')
Theta=np.load('ThetamatrixTest.npy')

# dimension
n_users = Y.shape[1]
n_movies = Y.shape[0]
n_features = X.shape[1]

### Task 8
***
Call cofiCostFunc with lamb=0 (without regularization term) and check the result

In [125]:
# Evaluate Cost J (without regularization term)
J=0
parameters=np.append(X.flatten(),Theta.flatten())

# YOUR CODE ..................................
# call cofiCostFunc with lamb=0 (without regularization term)


J = cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, 0)


# YOUR CODE (end)..................................

print('The value of J (without regularization term) is %0.2f (it should be 22.22)' % J[0] )

The value of J (without regularization term) is 22.22 (it should be 22.22)


### Task 9
***
Call cofiCostFunc with lamb=1.5 (with regularization term) and check the result

In [126]:
# Evaluate Cost J (with regularization term)
J=0
parameters=np.append(X.flatten(),Theta.flatten())

# YOUR CODE ..................................
# call cofiCostFunc with lamb=1.5 (without regularization term)

J = cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, 1.5)



# YOUR CODE (end)..................................

print('The value of J (with regularization term equal to 1.5) is %0.2f (it should be 31.34)' % J[0] )

The value of J (with regularization term equal to 1.5) is 31.34 (it should be 31.34)


### 2.4 Computing the gradient of J
The **gradient of J** depends on the two types of parameters, i.e. X and Theta. The corresponding equations are:

$$\frac { \partial J }{ \partial { \theta  }_{ k }^{ (j) } } =\sum _{ i:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) { x }_{ k }^{ (i) } } +\lambda { \theta  }_{ k }^{ (j) }$$

$$\frac { \partial J }{ \partial { x }_{ k }^{ (i) } } =\sum _{ j:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) \theta _{ k }^{ (j) } } +\lambda { x }_{ k }^{ (i) }$$

### Task 10
***
Now, go back and **complete the cofiCostFunc code to compute the gradient of J**. Remember to use vectorized operations instead of using for loops.

Note that the outputs of cofiCostFunc are the cost J (scalar value) and the gradient, again a **flattened vector of the corresponding gradients of X and Theta**:

$${ \left[ \begin{matrix} \frac { \partial J }{ \partial { x }^{ (1) } } , & ... & \frac { \partial J }{ \partial { x }^{ (m) } } , & \frac { \partial J }{ \partial \theta ^{ (1) } } , & ... & \frac { \partial J }{ \partial \theta ^{ (u) } }  \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$

After computing both gradients, you should reshape them into a flattened vector called **gradient** that will be returned by the cofiCostFunc.

### 2.5 Checking the gradient of J
For the same dataset of the last poit, you will check the gradient of J computed by your cofiCostFunc

In [127]:
# Check gradients (without regularization term) by running the next function
checkCostFunction(cofiCostFunc,0)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(1.3087481546780744, 1.3087481546779531)
(1.0563844320055082, 1.0563844320060467)
(1.0412680066052005, 1.0412680066060844)
(0.08519569861586973, 0.08519569861634141)
(0.23891556311772533, 0.2389155631176723)
(0.08277845564674813, 0.08277845564646907)
(1.1314171195753175, 1.131417119576377)
(1.152691331930411, 1.1526913319292933)
(0.8986444990566334, 0.8986444990564326)
(-0.11587720785688482, -0.11587720785577697)
(-0.8277296344427754, -0.8277296344438764)
(-0.2770643680394258, -0.27706436803956896)
(-0.20481966876539914, -0.2048196687650042)
(-0.017485102089542792, -0.01748510208932063)
(-0.06679118704222553, -0.06679118704214843)
(0.20154629868596174, 0.20154629868615184)
(0.563314248949176, 0.563314248949688)
(0.4523965958502796, 0.45239659585079717)
(-0.37960295849570613, -0.37960295849609677)
(-0.032406050272104636, -0.03240605027209413)
(-0.12378758522335609, -0.123

In [128]:
# Check gradients (with regularization term) by running the next function
checkCostFunction(cofiCostFunc,1.5)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(1.5196799849270803, 1.5196799849164193)
(1.0035059882618214, 1.003505988268374)
(1.5380143311816141, 1.5380143311782573)
(0.5508999751846488, 0.5508999751890699)
(1.0964159858684042, 1.09641598586486)
(0.3623674398411936, 0.36236743984618025)
(1.7855456340498677, 1.7855456340469777)
(2.233390518080469, 2.2333905180828864)
(2.0251683521621544, 2.025168352153033)
(1.5086402785335906, 1.5086402785349546)
(0.8754609098815536, 0.8754609098783597)
(2.0620686829797563, 2.0620686829838033)
(2.1287211394493966, 2.1287211394512453)
(1.155537652710592, 1.155537652709229)
(2.059256499249429, 2.0592564992507008)
(0.6798053492040879, 0.6798053492028915)
(1.1182202000359354, 1.1182202000354713)
(1.6615340387549082, 1.661534038755761)
(1.3341063731786562, 1.334106373183573)
(1.1453095777369526, 1.145309577729688)
(1.0939961656930564, 1.0939961656874717)
(0.9742896785613908, 0.974289678

## 3 Learning and recommendation
Finally, you will use your cofiCostFun to make predictions using the initial Movielens dataset. Part of the python code you need is already written in the next cells. You only have to complete those lines that are explicitly required.

### Task 11
***
Load the matrix Y and R computed in the first notebook.

In [129]:
# Task: load matrix Y and R
# YOUR CODE ..................................


Y = np.load('matrizy.npy')
R = np.load('matrizr.npy')


### Task 12
***
* Get the number of users and movies and assign the corresponding variables n_users, n_movies.
* Set the initial parameters (Theta, X) with random values.
* Fold X and Theta into the variable initial_parameters.

In [130]:
# Set the number of features
n_features = 100
n_users = Y.shape[1]
n_movies = Y.shape[0]
print(n_users)
print(n_movies)

X = np.random.random_sample((Y.shape[0],n_features))
Theta = np.random.random_sample((Y.shape[1],n_features))

initial_parameters = np.append(X.flatten(),Theta.flatten())

945
1682


Now, we set the rest of the parameters and minimize the function

In [131]:
# Set the regularization parameter
lamb = 10

# Define a function to be minimized
def cofiCostFunc_minimize(parameters):
    return cofiCostFunc(parameters,Y, R, n_users, n_movies, n_features,lamb)

# Set the number of iteations
max_iter=200

In [132]:
# Minimize the function using minimize from the package scipy.optimize and get the optimized parameters
parameters = (minimize(cofiCostFunc_minimize,initial_parameters,method="CG",jac=True,
                   options={'maxiter':max_iter, "disp":True})).x

         Current function value: 66608.791116
         Iterations: 200
         Function evaluations: 298
         Gradient evaluations: 298


### Task 13
***
Get the matrix of predictions P


In [133]:
# YOUR CODE ..................................
P = np.dot(X,Theta.T)

### Task 14
***
Show the titles of the top-5 predictions for the first user u=0, for those films user u did not watch: r(i,u)=0 (they will be the top-5 recommendations)

#### Tips
* You can import movies' titles using Pandas (see the first notebook)


In [134]:
# YOUR CODE ..................................

from pandas import read_table
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
items.drop(range(2,24),axis=1, inplace=True)
items.columns = ['itemid','title']

recomendacion_user0 = P[:,0]
no_vistas = np.where(R[0]==0)[0]
prediccion = recomendacion_user0[no_vistas]
top_5 = np.argsort(prediccion)[0:5]

for i in top_5:
    print(items.iloc[i]["title"])





Independence Day (ID4) (1996)
Four Weddings and a Funeral (1994)
Usual Suspects, The (1995)
Sense and Sensibility (1995)
Supercop (1992)


In [135]:
# Recomendaciones para nuestro usuario 944

from pandas import read_table
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
items.drop(range(2,24),axis=1, inplace=True)
items.columns = ['itemid','title']

recomendacion_user944 = P[:,943]
no_vistas = np.where(R[943]==0)[0]
prediccion = recomendacion_user944[no_vistas]
top_10 = np.argsort(prediccion)[0:10]

for i in top_10:
    print(items.iloc[i]["title"])

101 Dalmatians (1996)
This Is Spinal Tap (1984)
Carrington (1995)
Ulee's Gold (1997)
Primal Fear (1996)
English Patient, The (1996)
Godfather, The (1972)
Return of the Jedi (1983)
Brothers McMullen, The (1995)
Chasing Amy (1997)


In [136]:
# Implementar una función que teniendo en cuenta el filtro entrenado, calcule una distancia entre dos usuarios cualquiera
# Para realizar este apartado haremos la distancia euclidea que es la que hemos visto en Computacion Neuronal y Evolutiva

def euclidea(user1,user2):
    return np.sqrt(np.sum(Theta[user1-1,:] - Theta[user2-1,:])**2)

In [137]:
# Obtener el usuario más próximo a nosotros

usuarios_indice = range(1,n_users+1)

distancia_euclidea = []

for i in usuarios_indice:
    if i != 944:
        distancia_euclidea.append(euclidea(i,944))
usuario_mas_proximo = distancia_euclidea.index(min(distancia_euclidea)) + 1
print(usuario_mas_proximo)

614


In [138]:
# Implementar una función que teniendo en cuenta el filtro entrenado, calcule una distancia entre dos películas cualquiera.
# Para realizar este apartado haremos la distancia euclidea que es la que hemos visto en Computacion Neuronal y Evolutiva

def euclidea_peliculas(peli1,peli2):
    return np.sqrt(np.sum(X[peli1-1,:] - X[peli2-1,:])**2)

In [139]:
# Obtener las dos películas más próximas entre sí, y las dos más lejanas de la base de datos de Movielens.

peliculas_indice = range(1, n_movies+1)
min_rating = 10
max_rating = 0
pelis_cercanas = []
pelis_lejanas = []

for i in peliculas_indice:
    for j in peliculas_indice:
        if i != j:
            distancia_euclidea = euclidea_peliculas(i,j)
            
            if distancia_euclidea < min_rating:
                min_rating = distancia_euclidea
                pelis_cercanas = [i,j]
            if distancia_euclidea > max_rating:
                max_rating = distancia_euclidea
                pelis_lejanas = [i,j]
                
print(pelis_cercanas)
print(euclidea_peliculas(pelis_cercanas[0],pelis_cercanas[1]))
print(pelis_lejanas)
print(euclidea_peliculas(pelis_lejanas[0],pelis_lejanas[1]))

[326, 579]
6.238947222736435e-06
[236, 1122]
18.24927435592221


In [141]:
# Añadir un nuevo usuario sin ratings, y obtener un top-10 de recomendaciones de películas

from pandas import read_table
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
items.drop(range(2,24),axis=1, inplace=True)
items.columns = ['itemid','title']

recomendacion_user945 = P[:,944]
no_vistas = np.where(R[944]==0)[0]
prediccion = recomendacion_user945[no_vistas]
top_10 = np.argsort(prediccion)[0:10]

for i in top_10:
    print(items.iloc[i]["title"])

Room with a View, A (1986)
Star Trek III: The Search for Spock (1984)
Candidate, The (1972)
Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
Wild Bunch, The (1969)
Net, The (1995)
Remains of the Day, The (1993)
Army of Darkness (1993)
Once Upon a Time in the West (1969)
Police Story 4: Project S (Chao ji ji hua) (1993)
